<a href="https://colab.research.google.com/github/MuzaffarAli13/OpenAI-Agents-Sdk-Topics-Governor-House/blob/main/Agent_Hooks_8_August_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install openai-agents SDK


In [ ]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00


## Make your Notebook capable of running asynchronous functions.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Config

In [ ]:
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
)
from google.colab import userdata

In [ ]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)


In [ ]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

## Learning Agent LifeCycle

In [ ]:
# Imports
import asyncio
import random
from typing import Any

from pydantic import BaseModel

from agents import Agent, RunContextWrapper, AgentHooks, Runner, Tool, function_tool


In [ ]:

# Base model for context
class Order(BaseModel):
    order_id: str
    customer_name: str
    amount: float

# Simple Agent Hooks - track what one agent does
class SimpleAgentHooks(AgentHooks):
    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        """The agent is starting"""
        # print(f"[START] Agent '{agent.name}' started.")
        print("agent started during class")

    async def on_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        """This agent finished"""
        print(f"[END] Agent '{agent.name}' finished.")

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        """The agent is using tool"""
        # print(f"[TOOL START] Agent '{agent.name}' is using tool '{tool.name}'.")
        print("refund order tool started during class")

    async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool: Tool, result: str) -> None:
        """The agent's tool finished'"""
        # print(f"[TOOL END] Tool '{tool.name}' done. Result: {result}")
        print("refund order tool finished during class")


In [ ]:
# Simple tool - uses context wrapper
@function_tool
def refund_order(wrapper: RunContextWrapper[Order]):
    """Process a refund for the customer order"""
    print("refund_order tool is being called.")
    order = wrapper.context
    return f"Refund of ${order.amount} processed for order {order.order_id} (Customer: {order.customer_name})"

# Create hooks for our agent
my_hooks = SimpleAgentHooks()

# Create agent with hooks attached
support_agent = Agent(
    name="Support Bot",
    instructions="""You are a customer support agent. You must use the refund_order tool to process refunds.
    IMPORTANT: Always call the refund_order tool when someone ask about a refund. Do not just say you'll process it actually use the tool""",
    tools=[refund_order],
    model=model,
    hooks=my_hooks # <- HOOKS ATTACHED TO THIS AGENT
)



In [ ]:
async def main():

    print('=' * 50)
    print("AGENT HOOKS DEMO")
    print('=' * 50)

    # Create order context
    order = Order(
        order_id="ORD-12345",
        customer_name="Jhon Doe",
        amount=99.99,
    )

    # Run the agent with context
    result = await Runner.run(
        starting_agent=support_agent,
        input="I need a refund for my order. Please process my refund",
        context=order,
    )

    print(f"\n[ANSWER] {result.final_output}")

asyncio.run(main())


AGENT HOOKS DEMO
[START] Agent 'Support Bot' started.
[TOOL START] Agent 'Support Bot' is using tool 'refund_order'.
refund_order tool is being called.
refund order tool finished during class
[END] Agent 'Support Bot' finished.

[ANSWER] OK. I have processed a refund of $99.99 for your order. The order number is ORD-12345 and the customer name is Jhon Doe.

